In [5]:
import MDAnalysis as mda
import numpy as np

In [6]:
# create a universe from the dump file
# guess bond based on distance from the initial topology
u = mda.Universe("cnt_atom.data", "dump.lammpstrj",
                 topology_format="data", format="lammpsdump",
                 guess_bonds=True, vdwradii={'1':1.7})
# create a group
cnt = u.select_atoms("type 1")

In [7]:
nbond_vs_time = []
lbond_vs_time = []
# loop over trajectory
for ts in u.trajectory:
    # sabe the bond of the timestep ts in a list
    all_bonds_ts = []
    # loop over all initially detected bond
    for id1, id2 in cnt.atoms.bonds.indices:
        # detect positions
        pos1 = u.atoms.positions[u.atoms.indices == id1]
        pos2 = u.atoms.positions[u.atoms.indices == id2]
        d = pos1-pos2
        r = np.sqrt(d[:, 0]**2 + d[:, 1]**2 + d[:, 2]**2)
        if r < 1.8: # assume that bond longer than 1.8 angstroms are broken
            all_bonds_ts.append(r)
    lbond_vs_time.append([ts.time*5000*0.0005, np.mean(all_bonds_ts)]) 
    nbond_vs_time.append([ts.time*5000*0.0005, len(all_bonds_ts)/2]) # divide by 2 to avoid counting twice
nbond_vs_time = np.array(nbond_vs_time)
lbond_vs_time = np.array(lbond_vs_time)

In [8]:
np.savetxt("number_bond_vs_time.dat", nbond_vs_time)
np.savetxt("length_bond_vs_time.dat", lbond_vs_time)